In [1]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Create Train and Target Data

In [7]:
# get codes
gallery_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_gallery_codes', mode='rb'))
probe_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_probe_codes', mode='rb'))
# feature shape
print("gallery length: {}".format(gallery_features_batch1.shape))
print("gallery shape: {}".format(gallery_features_batch1[0].shape))
print("")
print("probe length: {}".format(probe_features_batch1.shape))
print("probe shape: {}".format(probe_features_batch1[0].shape))

gallery length: (93,)
gallery shape: (32, 1000)

probe length: (93,)
probe shape: (31, 1000)


### Add Zero

In [37]:
gallery_features_batch1_add0 = np.zeros([len(gallery_features_batch1), 43, 1000], dtype=float)
probe_features_batch1_add0 = np.zeros([len(probe_features_batch1), 43, 1000], dtype=float)

g_seq_length = []
p_seq_length = []
for ii, (gallery, probe) in enumerate(zip(gallery_features_batch1, probe_features_batch1)):
    gallery_length = gallery.shape[0]
    probe_length = probe.shape[0]
    g_seq_length.append(gallery_length)
    p_seq_length.append(probe_length)
    
    gallery_features_batch1_add0[ii, :gallery_length, :] = gallery
    probe_features_batch1_add0[ii, :probe_length, :] = probe

### Create Target 1 Data

In [10]:
batch_1_target_1 = np.concatenate((gallery_features_batch1_add0, probe_features_batch1_add0), axis=1)
target_1 = np.ones([batch_1_target_1.shape[0]])

In [11]:
print("batch_1_target_1:", batch_1_target_1.shape)
print("")
print("target_1:", target_1.shape)

batch_1_target_1: (93, 86, 1000)

target_1: (93,)


### Create Target 0 Data
We create a same length batch with shuffled data
直到检测shuffle的idx没有重叠，函数会返回ok，否则打印error

In [20]:
idx = np.arange(len(gallery_features_batch1_add0))
np.random.shuffle(idx)
idx

array([28, 90, 79, 44, 41, 16, 38, 52, 31, 36, 78, 51, 73, 49, 25,  7, 12,
       57, 62, 17, 80, 55, 48, 74, 14, 89,  6, 40, 42, 37,  5,  0, 81,  9,
        2, 75, 43, 58, 54,  4, 32, 77, 34, 56, 82, 88, 76,  8, 61, 53, 23,
       72, 15, 35, 70, 68, 50, 29, 60, 67, 20, 30, 66, 11, 69, 71, 84, 85,
       18, 64, 47, 27, 92, 39, 87, 19, 10, 46, 45, 22, 33, 59,  1, 86, 83,
       13,  3, 24, 65, 91, 63, 21, 26])

In [21]:
def exam_shuffle_idx(idx):
    x =None
    for ii, index in enumerate(idx):
        if ii == index:
            x ='error'
            print(x)
            break
    if x != 'error':
        print("ok")
    return None
exam_shuffle_idx(idx)

ok


In [22]:
# create shuffled half_probe_features_batch1
shuffled_probe_features_batch1 = np.zeros(probe_features_batch1_add0.shape)
print(shuffled_probe_features_batch1.shape)
for ii, index in enumerate(idx):
    shuffled_probe_features_batch1[ii, :, :] = probe_features_batch1_add0[index]

(93, 43, 1000)


In [23]:
batch_1_target_0 = np.concatenate((gallery_features_batch1_add0, shuffled_probe_features_batch1), axis=1)
target_0 = np.zeros([batch_1_target_0.shape[0]])

In [24]:
batch_1_target_0[0][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
batch_1_target_1[28][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
# Concatenate half_batch_1_target_0 and half_batch_1_target_1 together
new_batch_1 = np.concatenate((batch_1_target_1, batch_1_target_0), axis=0)
new_batch_1.shape

(186, 86, 1000)

In [27]:
with open('gait_data/fc8_90/ordered_new_batch_1', 'wb') as f:
    np.save(f, new_batch_1)

In [29]:
target_1_and_0 = np.concatenate((target_1, target_0))

In [30]:
# Shuffle new_half_batch_1 to random order
idx2 = np.arange(len(new_batch_1))
np.random.shuffle(idx2)
idx2

array([ 78,  31,  59,  42,  68,  28, 180,  93, 162,  24,  49, 135,  47,
         7, 182,  40, 167,  43, 116, 169, 104,   6, 181,  62,  84, 145,
       138, 161, 185,  16, 105, 140, 121,  45, 110, 141, 125, 164, 152,
       132, 143,  71, 174,  46,   5, 112,  32, 156, 124,  76,  60, 160,
       113,  27, 147,  91, 111,  38,  29,  26, 114, 102,  70,  33,  12,
        10,  54, 165,  57,  25, 131,  92, 158, 144,   0,  82, 126,  17,
       146, 119, 148,  58,   4, 100, 133,  19, 170, 150, 115,  21,  67,
        94,  99,  48, 177, 101, 108,  98,  73,  83,  95,  20,  13, 106,
        34, 153,  14, 123,  18, 171,  81,  36,  22,  86, 134, 139, 184,
       151,   3, 129, 179,  97, 127, 107,   2, 157,  37, 172,  23, 178,
        96,  69, 149,  39,  77, 154, 103,  90,  44,  55,  65,  53,  11,
         9,   8,  56,  85,  35,  51, 166,  41,  52,  75,  89, 175, 173,
       142,   1, 176,  30,  79,  64, 168,  61, 120, 128, 130,  63,  80,
        50, 163,  15,  74, 117, 155, 183, 109,  66, 118,  88,  7

In [31]:
shuffled_new_batch_1 = np.ndarray(new_batch_1.shape, dtype=float)
shuffled_target_1_and_0 = np.ndarray(target_1_and_0.shape, dtype=float)
for ii, index in enumerate(idx2):
    shuffled_new_batch_1[ii, :, :] = new_batch_1[index]
    shuffled_target_1_and_0[ii] = target_1_and_0[index]

In [32]:
with open('gait_data/fc8_90/shuffled_new_batch_1', 'wb') as f:
    np.save(f, shuffled_new_batch_1)

In [33]:
one_hot_shuffled_new_batch_1_target = np.ndarray([186, 2], dtype=float)
for ii, each in enumerate(shuffled_target_1_and_0):
    if int(each) == 1:
        one_hot_shuffled_new_batch_1_target[ii, :] = 1, 0
    elif int(each) == 0:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0, 1

In [34]:
with open('gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', 'wb') as f:
    np.save(f, one_hot_shuffled_new_batch_1_target)

## Create seq_length

In [38]:
p_seq_length = np.array(p_seq_length)
shuffled_p_seq_length = np.zeros(p_seq_length.shape, dtype=int)
for ii, index in enumerate(idx):
    shuffled_p_seq_length[ii] = p_seq_length[index]

In [39]:
seq_column1 = np.reshape(np.concatenate([g_seq_length, g_seq_length]), [186, 1])
seq_column2 = np.reshape(np.concatenate([g_seq_length, shuffled_p_seq_length]), [186, 1])
seq_length = np.concatenate([seq_column1, seq_column2], axis=1)

shuffled_seq_length = np.zeros(seq_length.shape, dtype=int)
for ii, index in enumerate(idx2):
    shuffled_seq_length[ii] = seq_length[index]

In [40]:
with open('gait_data/fc8_90/shuffled_seq_length', 'wb') as f:
    np.save(f, shuffled_seq_length)

# Checkpoints 1

In [110]:
seq_length = np.load(open(r'gait_data/fc8_90/shuffled_seq_length', mode='rb'))
shuffled_new_batch_1 = np.load(open(r'gait_data/fc8_90/shuffled_new_batch_1', mode='rb'))
one_hot_shuffled_new_batch_1_target = np.load(open(r'gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', mode='rb'))
print(shuffled_new_batch_1.shape)

(186, 86, 1000)


# Get Train and Validation data

In [111]:
print("data length: {}".format(len(shuffled_new_batch_1)))
print("every length has : {}".format(shuffled_new_batch_1.shape))

data length: 186
every length has : (186, 86, 1000)


In [112]:
split_frac = 0.8
split_num = int(split_frac*len(shuffled_new_batch_1))

train_x_raw, val_x_raw = shuffled_new_batch_1[:split_num], shuffled_new_batch_1[split_num:]
train_y_raw, val_y_raw = one_hot_shuffled_new_batch_1_target[:split_num], one_hot_shuffled_new_batch_1_target[split_num:]
train_seq_length, val_seq_length = seq_length[:split_num], seq_length[split_num:]

# print size
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))
print("train_seq_length: {}".format(np.array(train_seq_length).shape))
print("val_seq_length: {}".format(np.array(val_seq_length).shape))

Train_x: (148, 86, 1000)
Validation_x: (38, 86, 1000)
Train_y: (148, 2)
Validation_y: (38, 2)
train_seq_length: (148, 2)
val_seq_length: (38, 2)


# According_len_get_outputs

In [113]:
def According_len_get_outputs(length_batch_1, length_batch_2, outputs_1, outputs_2):
    """
    length_batch_1 and length_batch_2 should minus one
    """
    new_outputs_1 = np.ndarray([outputs_1.shape[0], outputs_1.shape[2]], dtype=float)
    new_outputs_2 = np.ndarray([outputs_2.shape[0], outputs_2.shape[2]], dtype=float)
    
    for ii, (L1, L2, O1, O2) in enumerate(zip(length_batch_1, length_batch_2, outputs_1, outputs_2)):
        new_outputs_1[ii, :] = O1[L1, :]
        new_outputs_2[ii, :] = O2[L2, :]
        
    return new_outputs_1, new_outputs_2

# Build the graph

In [114]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, grad_clip=1):
    
    # reset graph
    tf.reset_default_graph()
    
    with tf.name_scope('inputs_1'):
        inputs_1 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_1")
    with tf.name_scope('inputs_2'):
        inputs_2 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_2")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, 2], name="targets")
        
    seq_1 = tf.placeholder(tf.float32, (None,), name="seq_1")
    seq_2 = tf.placeholder(tf.float32, (None,), name="seq_2")
    
    new_outputs_1 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_1")
    new_outputs_2 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_2")
    
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
#     seq_length = tf.placeholder(tf.int32, (None, ), name='seq_length')
    with tf.variable_scope("LSTM"):
#         with tf.name_scope("RNN_cells_1"):
            # Your basic LSTM cell
        #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
        #         lstm = tf.nn.relu(lstm)
        lstm_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_1 = tf.contrib.rnn.DropoutWrapper(lstm_1, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_1 = tf.contrib.rnn.MultiRNNCell([drop_1] * num_layers)

#         with tf.name_scope("RNN_init_state_1"):
            # Getting an initial state of all zeros
        initial_state_1 = cell_1.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_1"):
        outputs_1, final_state_1 = tf.nn.dynamic_rnn(cell_1, inputs_1, initial_state=initial_state_1, 
                                                     sequence_length=seq_1)

    with tf.variable_scope("LSTM", reuse=True):
#         with tf.name_scope("RNN_cells_2"):
            # Your basic LSTM cell
    #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
    #         lstm = tf.nn.relu(lstm)
        lstm_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_2 = tf.contrib.rnn.DropoutWrapper(lstm_2, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_2 = tf.contrib.rnn.MultiRNNCell([drop_2] * num_layers)

#         with tf.name_scope("RNN_init_state_2"):
            # Getting an initial state of all zeros
        initial_state_2 = cell_2.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_2"):
        outputs_2, final_state_2 = tf.nn.dynamic_rnn(cell_2, inputs_2, initial_state=initial_state_2, 
                                                     sequence_length=seq_2)

    with tf.name_scope('predictions'):
        # only choose final one
#         final_output_concat = tf.concat([outputs_1[:, -1], outputs_2[:, -1]], 1)

        # through the According_len_get_outputs
    
#         # subtraction
#         final_output_concat = new_outputs_1 - new_outputs_2 
        
        # concatenation
        final_output_concat = tf.concat([new_outputs_1, new_outputs_2], 1)
        
        logits = tf.contrib.layers.fully_connected(final_output_concat, 2)
        predictions = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', predictions)
        
    with tf.name_scope('cost'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_, logits=logits))
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
#         tvars = tf.trainable_variables()
#         grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
#         train_op = tf.train.AdamOptimizer(learning_rate)
#         optimizer = train_op.apply_gradients(zip(grads, tvars))
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_1', 'inputs_2', 'target_', 'initial_state_1', 'initial_state_2', 'outputs_1', 'outputs_2',
                    'final_state_1', 'final_state_2', 'keep_prob', 'cost', 'logits', 'predictions', 'optimizer', 
                    'merged', 'saver', 'learning_rate', 'cell_1', 'cell_2', 'final_output_concat', 'seq_1', 'seq_2',
                    'new_outputs_1', 'new_outputs_2']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Get Batches

In [115]:
def get_batches(data, target, seq_length, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        seq_length_batch_1 = seq_length[ii:ii + batch_size, 0]
        seq_length_batch_2 = seq_length[ii:ii + batch_size, 1]
        
        yield data_batch, target_batch, seq_length_batch_1, seq_length_batch_2

# Accuracy

In [116]:
train_seq_length

array([[28, 28],
       [31, 31],
       [30, 30],
       [33, 33],
       [30, 30],
       [31, 31],
       [30, 32],
       [32, 31],
       [26, 33],
       [32, 32],
       [27, 27],
       [33, 34],
       [32, 32],
       [31, 31],
       [31, 34],
       [32, 32],
       [31, 31],
       [30, 30],
       [33, 30],
       [30, 32],
       [32, 32],
       [35, 35],
       [30, 30],
       [29, 29],
       [31, 31],
       [23, 33],
       [25, 31],
       [30, 32],
       [28, 32],
       [33, 33],
       [30, 32],
       [32, 30],
       [31, 35],
       [25, 25],
       [31, 29],
       [29, 30],
       [32, 31],
       [31, 32],
       [30, 31],
       [31, 33],
       [28, 33],
       [31, 31],
       [31, 27],
       [31, 31],
       [31, 31],
       [30, 31],
       [32, 32],
       [30, 33],
       [31, 31],
       [30, 30],
       [30, 30],
       [31, 30],
       [30, 28],
       [32, 32],
       [29, 32],
       [34, 34],
       [31, 30],
       [33, 33],
       [29, 29

# Train

In [117]:
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [118]:
train_x.shape

(148, 86, 1000)

In [119]:
def train(model, epochs, file_writer, save_string):

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        for e in range(epochs):
            for train_batch, target_batch, seq_length_batch_1, seq_length_batch_2 \
                             in get_batches(train_x, train_y, train_seq_length, batch_size):
                start = time.time()
#                 state_1 = sess.run(model.initial_state_1)
#                 state_2 = sess.run(model.initial_state_2)
#                 print(train_batch[:, :15, :].shape)
#                 print(target_batch)
    #             print('target: {}'.format(target_batch.shape))
                
                feed_1 = {
                        model.inputs_1: train_batch[:, :43, :],
                        model.inputs_2: train_batch[:, 43:, :],  
                        model.seq_1: seq_length_batch_1,
                        model.seq_2: seq_length_batch_2,
                        model.keep_prob: 1
                        }
            
#                 print(model.outputs_1.shape, model.outputs_1[:, -1], model.final_output_concat, 
#                       model.logits, model.predictions, model.cost)
                o_1, o_2 = sess.run([model.outputs_1, model.outputs_2], feed_dict=feed_1)
                
                real_outputs_1, real_outputs_2 = According_len_get_outputs(
                                               seq_length_batch_1 - 1, seq_length_batch_2 - 1, o_1, o_2)
#                 print(o_1[:, 28], o_2[:, 28])
#                 print(real_outputs_1, real_outputs_2)
                feed_2 = {
                        model.target_: target_batch,
#                         model.keep_prob: 1,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                        model.learning_rate: lr,
                        model.new_outputs_1: real_outputs_1,
                        model.new_outputs_2: real_outputs_2
                        }
                
                loss, _, logi, pred, summary,  = sess.run(
                                [model.cost, model.optimizer, model.logits, model.predictions, model.merged]
                                , feed_dict=feed_2)
                print(pred)
#                 print(logi)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
                
                if iteration%5==0:
                    end = time.time()
#                     acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print("Epoch: {}/{},".format(e+1, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' '
#                           "ACCuracy: %{:.3f}".format(acc)
                         )
                    
                    file_writer.add_summary(summary, iteration)
                
#                 if iteration%25==0:
#                     validation_loss = []
#                     validation_Acc = []
                    
#                     if batch_size >= len(val_x):
#                         val_batch_size = 30
#                     else: 
#                         val_batch_size = batch_size
                    
#                     for val_batch, val_target_batch in get_batches(val_x, val_y, val_batch_size):

#                         val_state_1 = sess.run(model.cell_1.zero_state(val_batch_size, tf.float32))
#                         val_state_2 = sess.run(model.cell_2.zero_state(val_batch_size, tf.float32))

#                         feed = {model.inputs_1: val_batch[:, :15, :],
#                                 model.inputs_2: val_batch[:, 15:, :],
#                                 model.target_: val_target_batch,
#                                 model.keep_prob: 1,
#                                 model.initial_state_1: val_state_1,
#                                 model.initial_state_2: val_state_2,
#                                }

#                         val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed)
                        
# #                         val_acc = calculate_accuracy(sess, val_pred, val_target_batch, val_sq_length)

#                         validation_loss.append(val_loss)
                        
# #                         validation_Acc.append(val_acc)
                        
#                     Last_val_Acc = mean_val_Acc
#                     Last_val_loss = mean_val_loss
                        
#                     mean_val_loss = sum(validation_loss)/len(validation_loss)  
#                     mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
#                     print()
#                     print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
#                           "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
#                     print()
                    
                iteration += 1
                
            # Early stopping  
#             if mean_val_Acc - Last_val_Acc <= -0.3:
#                 count_Acc_not_increase_epochs += 1
#             if Last_val_loss - mean_val_loss <= -0.01:
#                 count_loss_not_decrease_epochs += 1
            
#             if count_Acc_not_increase_epochs >= 5:
#                 break
#             if count_loss_not_decrease_epochs >= 5:
#                 break
                
        model.saver.save(sess, r"{}".format(save_string))

In [120]:
!mkdir checkpoints_identification

mkdir: checkpoints_identification: File exists


In [121]:
epochs = 1000
batch_size = 100
for lstm_size in [3000]:
    for num_layers in [1]:
        for lr in [0.0005]:
                log_string = 'logs/4/lr={},nl={},ls={},bs={}'.format(lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\lr={}_nl={}_ls={}_bs={}.ckpt'.format(lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)

                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, 
                                  num_layers=num_layers, learning_rate=lr, grad_clip=100)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

[[ 0.50331169  0.49668837]
 [ 0.50471574  0.49528423]
 [ 0.49502885  0.50497115]
 [ 0.50416803  0.495832  ]
 [ 0.5010708   0.49892917]
 [ 0.50208986  0.49791008]
 [ 0.5         0.5       ]
 [ 0.49994349  0.50005656]
 [ 0.50161195  0.49838805]
 [ 0.49871805  0.50128192]
 [ 0.50395119  0.49604884]
 [ 0.5         0.5       ]
 [ 0.5013001   0.49869999]
 [ 0.5008375   0.4991625 ]
 [ 0.49892902  0.50107092]
 [ 0.49676579  0.50323421]
 [ 0.5055421   0.49445787]
 [ 0.50489622  0.49510387]
 [ 0.50210083  0.49789909]
 [ 0.5         0.5       ]
 [ 0.49999753  0.50000244]
 [ 0.5         0.5       ]
 [ 0.4956702   0.50432986]
 [ 0.50233489  0.49766511]
 [ 0.49982983  0.50017017]
 [ 0.51018447  0.4898155 ]
 [ 0.50201249  0.49798745]
 [ 0.50457138  0.49542868]
 [ 0.50178987  0.49821016]
 [ 0.50228655  0.49771348]
 [ 0.50425589  0.49574414]
 [ 0.50130916  0.49869078]
 [ 0.50733554  0.49266446]
 [ 0.49755993  0.50244009]
 [ 0.49661675  0.50338316]
 [ 0.49731573  0.50268418]
 [ 0.50109935  0.49890059]
 

[[ 0.52623051  0.47376946]
 [ 0.55510014  0.44489983]
 [ 0.56878102  0.43121892]
 [ 0.5         0.5       ]
 [ 0.49587435  0.50412565]
 [ 0.4950009   0.5049991 ]
 [ 0.49675289  0.5032472 ]
 [ 0.49255401  0.50744599]
 [ 0.49840936  0.50159061]
 [ 0.49097764  0.5090223 ]
 [ 0.49655941  0.5034405 ]
 [ 0.49617773  0.50382227]
 [ 0.49557513  0.50442487]
 [ 0.49278581  0.50721425]
 [ 0.48791578  0.51208419]
 [ 0.48615018  0.51384979]
 [ 0.5         0.5       ]
 [ 0.55403614  0.44596386]
 [ 0.48895371  0.51104629]
 [ 0.49291891  0.50708103]
 [ 0.49238324  0.50761676]
 [ 0.49312356  0.50687641]
 [ 0.48702702  0.51297295]
 [ 0.49257651  0.50742346]
 [ 0.49142584  0.50857413]
 [ 0.50010949  0.49989048]
 [ 0.49685752  0.50314248]
 [ 0.5         0.5       ]
 [ 0.49879041  0.50120962]
 [ 0.49299923  0.50700074]
 [ 0.5         0.5       ]
 [ 0.49450082  0.50549918]
 [ 0.5         0.5       ]
 [ 0.48971581  0.51028413]
 [ 0.48851234  0.51148766]
 [ 0.48842061  0.51157933]
 [ 0.49424011  0.50575989]
 

[[ 0.54798764  0.45201233]
 [ 0.59903544  0.40096462]
 [ 0.62782711  0.3721728 ]
 [ 0.49591956  0.50408041]
 [ 0.48810208  0.51189798]
 [ 0.48741531  0.51258475]
 [ 0.48961815  0.51038182]
 [ 0.48389745  0.51610255]
 [ 0.49172226  0.50827771]
 [ 0.48332319  0.51667678]
 [ 0.49082595  0.50917405]
 [ 0.48796532  0.51203471]
 [ 0.48769131  0.5123086 ]
 [ 0.48569238  0.51430762]
 [ 0.47928122  0.52071881]
 [ 0.47914168  0.52085835]
 [ 0.49471655  0.50528342]
 [ 0.59845275  0.40154728]
 [ 0.47981411  0.52018589]
 [ 0.48570174  0.51429826]
 [ 0.48361817  0.51638186]
 [ 0.48518622  0.51481384]
 [ 0.48022583  0.5197742 ]
 [ 0.48602873  0.51397127]
 [ 0.48340431  0.51659572]
 [ 0.5         0.5       ]
 [ 0.48900855  0.51099145]
 [ 0.49711722  0.50288278]
 [ 0.49136889  0.50863105]
 [ 0.48644811  0.51355195]
 [ 0.49277273  0.50722724]
 [ 0.4850933   0.5149067 ]
 [ 0.5         0.5       ]
 [ 0.4817802   0.51821977]
 [ 0.47995165  0.52004838]
 [ 0.47779822  0.52220178]
 [ 0.48679924  0.51320076]
 

[[ 0.56934661  0.43065342]
 [ 0.64057523  0.35942477]
 [ 0.68187112  0.31812891]
 [ 0.48776048  0.51223952]
 [ 0.4795824   0.52041763]
 [ 0.47861439  0.52138567]
 [ 0.48130837  0.5186916 ]
 [ 0.47436708  0.52563298]
 [ 0.48328635  0.51671368]
 [ 0.47493392  0.52506608]
 [ 0.4842304   0.51576954]
 [ 0.47909746  0.52090257]
 [ 0.47923765  0.52076232]
 [ 0.47807887  0.5219211 ]
 [ 0.47015756  0.5298425 ]
 [ 0.47227871  0.52772129]
 [ 0.48746884  0.5125311 ]
 [ 0.6405279   0.35947213]
 [ 0.47069728  0.52930272]
 [ 0.47780138  0.52219868]
 [ 0.4741312   0.52586877]
 [ 0.47694319  0.52305681]
 [ 0.473239    0.52676094]
 [ 0.47911999  0.52087998]
 [ 0.4752138   0.52478623]
 [ 0.49715009  0.50284994]
 [ 0.47977528  0.52022463]
 [ 0.48975202  0.51024795]
 [ 0.4826701   0.51732993]
 [ 0.47941977  0.52058017]
 [ 0.48361245  0.51638758]
 [ 0.47514668  0.52485335]
 [ 0.49419653  0.50580347]
 [ 0.4733991   0.5266009 ]
 [ 0.47103626  0.5289638 ]
 [ 0.46679744  0.53320253]
 [ 0.47821483  0.52178514]
 

[[ 0.59003484  0.40996519]
 [ 0.67907536  0.32092461]
 [ 0.72998905  0.27001092]
 [ 0.47839341  0.52160656]
 [ 0.47060221  0.52939779]
 [ 0.4689326   0.53106743]
 [ 0.47218779  0.52781218]
 [ 0.46448332  0.53551668]
 [ 0.47351265  0.52648741]
 [ 0.46608931  0.53391075]
 [ 0.47686392  0.52313614]
 [ 0.47016382  0.52983618]
 [ 0.47062314  0.52937686]
 [ 0.47006446  0.52993554]
 [ 0.46096122  0.53903872]
 [ 0.46562418  0.53437591]
 [ 0.47896633  0.5210337 ]
 [ 0.67953336  0.32046664]
 [ 0.46181726  0.53818274]
 [ 0.46951738  0.53048265]
 [ 0.46446121  0.53553879]
 [ 0.46863985  0.53136015]
 [ 0.46600196  0.53399807]
 [ 0.47189787  0.5281021 ]
 [ 0.46712652  0.53287351]
 [ 0.48806375  0.51193625]
 [ 0.46969539  0.53030455]
 [ 0.48133928  0.51866066]
 [ 0.47308624  0.52691382]
 [ 0.47193804  0.52806193]
 [ 0.47299427  0.52700573]
 [ 0.46514383  0.5348562 ]
 [ 0.48543224  0.51456779]
 [ 0.46493027  0.5350697 ]
 [ 0.46222052  0.53777957]
 [ 0.45615059  0.54384935]
 [ 0.4688158   0.53118426]
 

...

[[  9.66277540e-01   3.37224342e-02]
 [  9.98065650e-01   1.93441135e-03]
 [  9.99704182e-01   2.95768114e-04]
 [  3.67820889e-01   6.32179141e-01]
 [  3.77985269e-01   6.22014701e-01]
 [  2.86666572e-01   7.13333368e-01]
 [  4.08447713e-01   5.91552317e-01]
 [  3.15847963e-01   6.84152067e-01]
 [  2.83669233e-01   7.16330707e-01]
 [  3.62723947e-01   6.37276053e-01]
 [  5.00000000e-01   5.00000000e-01]
 [  4.17552173e-01   5.82447886e-01]
 [  4.67377335e-01   5.32622635e-01]
 [  4.19472337e-01   5.80527604e-01]
 [  3.38185012e-01   6.61814988e-01]
 [  4.99614835e-01   5.00385106e-01]
 [  3.12260747e-01   6.87739313e-01]
 [  9.98216808e-01   1.78322312e-03]
 [  3.71707469e-01   6.28292561e-01]
 [  3.25072855e-01   6.74927115e-01]
 [  3.90349239e-01   6.09650791e-01]
 [  4.08514053e-01   5.91485918e-01]
 [  3.87756169e-01   6.12243772e-01]
 [  4.87344533e-01   5.12655377e-01]
 [  4.81695056e-01   5.18304944e-01]
 [  3.12731892e-01   6.87268078e-01]
 [  3.00569981e-01   6.99429989e-01]
 

[[  9.66577351e-01   3.34225968e-02]
 [  9.98096764e-01   1.90326211e-03]
 [  9.99710262e-01   2.89705844e-04]
 [  3.67814243e-01   6.32185757e-01]
 [  3.78100306e-01   6.21899724e-01]
 [  2.86351591e-01   7.13648319e-01]
 [  4.08779323e-01   5.91220677e-01]
 [  3.15630525e-01   6.84369445e-01]
 [  2.83161163e-01   7.16838777e-01]
 [  3.62625659e-01   6.37374282e-01]
 [  5.00000000e-01   5.00000000e-01]
 [  4.17602777e-01   5.82397163e-01]
 [  4.67791349e-01   5.32208681e-01]
 [  4.19597566e-01   5.80402493e-01]
 [  3.38051379e-01   6.61948681e-01]
 [  4.99794632e-01   5.00205338e-01]
 [  3.11502904e-01   6.88497126e-01]
 [  9.98245955e-01   1.75407249e-03]
 [  3.71770829e-01   6.28229260e-01]
 [  3.24381143e-01   6.75618827e-01]
 [  3.90469968e-01   6.09530032e-01]
 [  4.08671826e-01   5.91328204e-01]
 [  3.87495130e-01   6.12504840e-01]
 [  4.87756819e-01   5.12243152e-01]
 [  4.82348740e-01   5.17651260e-01]
 [  3.12244833e-01   6.87755227e-01]
 [  3.00100058e-01   6.99899852e-01]
 

[[  9.66873050e-01   3.31269540e-02]
 [  9.98127162e-01   1.87278632e-03]
 [  9.99716222e-01   2.83801113e-04]
 [  3.67650956e-01   6.32349014e-01]
 [  3.78097445e-01   6.21902525e-01]
 [  2.85947025e-01   7.14052975e-01]
 [  4.08972651e-01   5.91027379e-01]
 [  3.15304279e-01   6.84695661e-01]
 [  2.82483816e-01   7.17516184e-01]
 [  3.62413198e-01   6.37586832e-01]
 [  5.00000000e-01   5.00000000e-01]
 [  4.17536288e-01   5.82463741e-01]
 [  4.68108326e-01   5.31891644e-01]
 [  4.19604450e-01   5.80395460e-01]
 [  3.37836236e-01   6.62163734e-01]
 [  4.99896616e-01   5.00103414e-01]
 [  3.10586482e-01   6.89413607e-01]
 [  9.98274446e-01   1.72555854e-03]
 [  3.71780008e-01   6.28220022e-01]
 [  3.23559314e-01   6.76440775e-01]
 [  3.90480757e-01   6.09519184e-01]
 [  4.08735096e-01   5.91264904e-01]
 [  3.87116045e-01   6.12883925e-01]
 [  4.88006741e-01   5.11993289e-01]
 [  4.82885420e-01   5.17114580e-01]
 [  3.11543912e-01   6.88456059e-01]
 [  2.99486876e-01   7.00513124e-01]
 

[[  9.67164576e-01   3.28354649e-02]
 [  9.98157084e-01   1.84296200e-03]
 [  9.99721944e-01   2.78048712e-04]
 [  3.67582947e-01   6.32417023e-01]
 [  3.78146350e-01   6.21853650e-01]
 [  2.85577953e-01   7.14422047e-01]
 [  4.09237474e-01   5.90762496e-01]
 [  3.15039456e-01   6.84960544e-01]
 [  2.81951547e-01   7.18048453e-01]
 [  3.62237900e-01   6.37762129e-01]
 [  5.00000000e-01   5.00000000e-01]
 [  4.17509049e-01   5.82491040e-01]
 [  4.68406230e-01   5.31593740e-01]
 [  4.19638664e-01   5.80361366e-01]
 [  3.37628394e-01   6.62371635e-01]
 [  4.99920547e-01   5.00079453e-01]
 [  3.09760660e-01   6.90239310e-01]
 [  9.98302341e-01   1.69766333e-03]
 [  3.71753693e-01   6.28246248e-01]
 [  3.22796643e-01   6.77203357e-01]
 [  3.90528172e-01   6.09471798e-01]
 [  4.08808947e-01   5.91191113e-01]
 [  3.86764258e-01   6.13235712e-01]
 [  4.88336593e-01   5.11663318e-01]
 [  4.83459443e-01   5.16540587e-01]
 [  3.11009347e-01   6.88990593e-01]
 [  2.98970133e-01   7.01029837e-01]
 

KeyboardInterrupt: 